This code is designed to optimize the diffusion weights in a Diffusion Kalman Filter (DKF) network by minimizing the Bhattacharyya distance between the distributions of the DKF estimates and those from a centralized Kalman Filter. The optimization process leverages the BFGS algorithm. By fine-tuning the diffusion weights, the goal is to achieve a set of weights that result in state estimates from the DKF that closely match those from the centralized approach, thereby improving the overall accuracy and reliability of the distributed state estimation process.

In [ ]:
!git clone https://github.com/RIPS-2024-Aerospace/Aerospace-Project.git

Cloning into 'Aerospace-Project'...
remote: Enumerating objects: 334, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (175/175), done.
remote: Total 334 (delta 107), reused 102 (delta 39), pack-reused 120
Receiving objects: 100% (334/334), 18.78 MiB | 10.28 MiB/s, done.
Resolving deltas: 100% (147/147), done.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(163)

# replace with file paths
%run "/content/Aerospace-Project/Standard Filters/DiffKf.ipynb"
%run "/content/Aerospace-Project/Standard Filters/KF.ipynb"
# %run "/content/Aerospace-Project/Standard Filters/DiffKf.ipynb"
# %run "/content/Aerospace-Project/Standard Filters/KF.ipynb"

The next few code blocks are taken directly from "testStandardOptimize.ipynb" to allow for modifying the code within this implementation.

In [ ]:
def run_filters(W):
  print(W)
  dt = 10

  C = np.array([[W[i] for i in range(0,5)], [W[i] for i in range(5,10)], [W[i] for i in range(10,15)], [W[i] for i in range(15,20)], [W[i] for i in range(20,25)]])

  C_unweighted = np.array([[1 if x!=0 else 0 for x in row] for row in C])

  num_stns = len(C[0])

  A = np.array([[1, dt, 0, 0], [0, 1, 0, 0],[0,0,1,dt], [0, 0, 0, 1]])
  H = np.array([[1, 0, 0, 0],[0,0,1,0]])

  dkf_state_size = len(A)
  dkf_measure_size = len(H)

  q = 0.001
  Q = q*np.array([[(dt**3)/3, (dt**2)/2, 0, 0], [(dt**2)/2, dt, 0, 0],[0,0,(dt**3)/3,(dt**2)/2], [0, 0, (dt**2)/2, dt]])
  R = np.array([[4,0],[0,4]])

  A_kf = np.kron(np.eye(num_stns),A)
  H_kf = np.kron(np.eye(num_stns),H)
  Q_kf = np.kron(np.eye(num_stns),Q)
  R_kf = np.kron(np.eye(num_stns),R)

  kf_state_size = A_kf.shape[0]
  kf_measure_size = R_kf.shape[0]

  F = [A for _ in range(num_stns)]
  G = [np.eye(dkf_state_size) for _ in range(num_stns)]
  H_dkf = [H for _ in range(num_stns)]

  Q_dkf = [Q for _ in range(num_stns)]
  R_dkf = [R for _ in range(num_stns)]

  procc_noise_kf = lambda : np.linalg.cholesky(Q_kf) @ np.random.normal(np.array([[0 for _ in range(kf_state_size)]]).T)
  measure_noise_kf = lambda : np.linalg.cholesky(R_kf) @ np.random.normal(np.array([[0 for _ in range(kf_measure_size)]]).T)

  measure_kf_to_dkf  = lambda z: [np.array([z[dkf_measure_size*i + j] for j in range(dkf_measure_size)]) for i in range(num_stns)]
  state_kf_to_dkf = lambda z: [np.array([z[dkf_state_size*i + j] for j in range(dkf_state_size)]) for i in range(num_stns)]

  #True Initial
  x0_kf = np.array([[np.random.normal(0,np.sqrt(Q_kf[i,i])) for i in range(kf_state_size)]]).T

  #Initial Estimate
  x_kf = np.array([[np.random.normal(0,5) for i in range(kf_state_size)]]).T
  x_dkf = state_kf_to_dkf(x_kf)


  P_kf = 10*np.copy(Q_kf)
  P_dkf = [10*np.copy(Q) for _ in range(num_stns)]

  kf = KalmanFilter(A = A_kf,H = H_kf, Q = Q_kf, R = R_kf,P=P_kf,x0=x_kf)

  dkf = DiffKF(C,F,G,H_dkf,R_dkf,Q_dkf,x_dkf,P_dkf)

  iters = 60

  truth = np.zeros((iters+1,kf_state_size,1))
  truth[0] = x0_kf

  measurements = np.zeros((iters+1,kf_measure_size,1))
  measurements[0] = (H_kf @ x0_kf)+measure_noise_kf()


  predictions_kf = np.zeros((iters,kf_state_size,1))
  predictions_dkf = np.zeros((iters,num_stns,dkf_state_size,1))

  errors_kf = np.zeros((iters,kf_state_size,1))
  errors_dkf = np.zeros((iters,num_stns,dkf_state_size,1))

  P_hist_kf = np.zeros((iters,kf_state_size,kf_state_size))
  P_hist_dkf = np.zeros((iters, num_stns, dkf_state_size,dkf_state_size))
  full_system_P_hist = np.zeros((iters,kf_state_size,kf_state_size))
  prev_cov = np.block([[np.zeros(P_dkf[0].shape) if i!= j else dkf.nodes[i].P for j in range(num_stns)] for i in range(num_stns)])


  for i in range(iters):

      kf.update(measurements[i])
      dkf.update(measure_kf_to_dkf(measurements[i]))

      predictions_dkf[i] = [dkf.nodes[j].x for j in range(num_stns)]
      errors_dkf[i] = [dkf.nodes[j].x-state_kf_to_dkf(truth[i])[j] for j in range(num_stns)]
      station_covs = [dkf.nodes[j].P for j in range(num_stns)]
      P_hist_dkf[i] = station_covs

      prev_cov = get_diff_cov(prev_cov, station_covs)
      full_system_P_hist[i] = prev_cov


      predictions_kf[i] = kf.x
      errors_kf[i] = kf.x-truth[i]
      P_hist_kf[i] = kf.P


      kf.predict()
      dkf.predict()

      truth[i+1] = A_kf@x0_kf + procc_noise_kf()
      measurements[i+1] = H_kf @ truth[i+1] + measure_noise_kf()

  return (P_hist_kf[40], full_system_P_hist[40])

In [ ]:
def get_diff_cov(prev_cov, Station_cov):

    S = lambda i: np.sum([node.H.T @ np.linalg.inv(node.R) @ node.H for node in dkf.nodes[i].nbhrs],axis = 0)

    S_full = np.block([[np.zeros(A.shape) if i!= j else S(j) for j in range(num_stns)] for i in range(num_stns)])
    H_full = np.kron(np.eye(num_stns),H)
    P_full = np.block([[np.zeros(P_dkf[0].shape) if i!= j else Station_cov[j] for j in range(num_stns)] for i in range(num_stns)])
    R_full = np.kron(np.eye(num_stns),R)


    C_full = np.kron(C,np.eye(dkf_state_size))
    A_full = np.kron(C_unweighted, np.eye(dkf_state_size))

    # Sigma1
    # compute the covariance (equation 32)
    F_i = C_full.T @ (np.eye(S_full.shape[1]) - (P_full @ S_full)) @ np.kron(np.eye(num_stns),A)
    G_i = C_full.T @ (np.eye(S_full.shape[1]) - (P_full @ S_full)) @ np.kron(np.eye(num_stns),G[0])
    D_i = C_full.T @ P_full @ A_full.T @ H_full.T @ np.linalg.inv(R_full)


    term1 = (F_i @ prev_cov @ F_i.T)
    term2 = G_i @ np.kron(np.ones((num_stns,num_stns)),Q)@G_i.T
    term3 = D_i @ R_full@D_i.T

    return term1 + term2 + term3

In [ ]:
def bhattacharyya_distance(mu1, mu2, Sigma1, Sigma2):
    # mu1 = mean of diffusion KF
    # mu2 = mean of centralized KF
    # Sigma1 = covariance of diffusion KF
    # Sigma2 = covariance of centralized KF
    Sigma = (Sigma1 + Sigma2) / 2
    inv_Sigma = np.linalg.inv(Sigma)

    term1 = 1/8 * np.dot(np.dot((mu1 - mu2).T, inv_Sigma), (mu1 - mu2))
    term2 = 1/2 * np.log(np.linalg.det(Sigma) / np.sqrt(np.linalg.det(Sigma1) * np.linalg.det(Sigma2)))

    return term1 + term2

This is where we import the BFGS method to optimize over weights from SciPy.

In [ ]:
import scipy as sp
from scipy.optimize import minimize

# Calculates the Bhattacharya Distance for the Centralized and Diffusion KFs.
def cost_func(diffusion_weights):
  mu1 = np.zeros(kf_state_size,int)
  mu2 = np.zeros(kf_state_size, int)
  sigmas = run_filters(diffusion_weights)
  Sigma1 = sigmas[0]
  Sigma2 = sigmas[1]
  return bhattacharyya_distance(mu1, mu2, Sigma1, Sigma2)

# Plugs in an initial set of weights before running BFGS
def run_optimize():
  C = np.array([[0.34,0.33, 0, 0, 0.33],[0.33,0.34,0.33,0,0],[0,0.33,0.34,0.33,0],[0,0,0.33,0.34,0.33],[0.33,0,0,0.33,0.34]])
  weights = []
  for arr in C:
    for val in arr:
      weights.append(val)

  x0 = np.array(weights)
  result = sp.optimize.minimize(cost_func, x0, method='BFGS')
  print(result.x)

In [ ]:
run_optimize()

[0.34 0.33 0.   0.   0.33 0.33 0.34 0.33 0.   0.   0.   0.33 0.34 0.33
 0.   0.   0.   0.33 0.34 0.33 0.33 0.   0.   0.33 0.34]
[0.34000001 0.33       0.         0.         0.33       0.33
 0.34       0.33       0.         0.         0.         0.33
 0.34       0.33       0.         0.         0.         0.33
 0.34       0.33       0.33       0.         0.         0.33
 0.34      ]
[0.34       0.33000001 0.         0.         0.33       0.33
 0.34       0.33       0.         0.         0.         0.33
 0.34       0.33       0.         0.         0.         0.33
 0.34       0.33       0.33       0.         0.         0.33
 0.34      ]


KeyboardInterrupt: 